<a href="https://colab.research.google.com/github/DvdNss/FaceMaskDetection/blob/main/Google_MT5_For_Everything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

# Install Repo
(do not forget to mount your drive to have the models stored in it automatically)

In [1]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [2]:
!git clone https://github.com/DvdNss/Google-MT5-For-Everything.git

fatal: destination path 'Google-MT5-For-Everything' already exists and is not an empty directory.


In [3]:
cd /content/drive/MyDrive/Google-MT5-For-Everything/

/content/drive/MyDrive/Google-MT5-For-Everything


In [4]:
!git pull

Already up to date.


In [ ]:
pip install -r requirements.txt

# Train

In [6]:
cd /content/drive/MyDrive/Google-MT5-For-Everything/

/content/drive/MyDrive/Google-MT5-For-Everything


In [ ]:
#@title Training Form
from source.datamodule import DataModule
from pytorch_lightning import seed_everything, Trainer
from source.mt5 import MT5
import os
import torch

train_file = 'data/train_example.tsv' #@param{type:'string'}
valid_file = 'data/valid_example.tsv' #@param{type:"string"}
inputs_col = 'inputs' #@param{type:"string"}
outputs_col = 'outputs' #@param{type:"string"}
tokenizer_name_or_path = "google/mt5-small" #@param{type:"string"}["google/mt5-small", "google/mt5-base", "google/mt5-large"]
input_max_length = 512 #@param{type:"slider", min:10, max:512}
output_max_length = 10 #@param{type:"slider", min:1, max:512}
batch_size = 1 #@param{type:"slider", min: 1, max:32}
output_dir = 'model' #@param{type:"string"}
model_name_or_path = 'google/mt5-small' #@param{type:"string"}["google/mt5-small", "google/mt5-base", "google/mt5-large"]
epochs = 20 #@param{type:"slider", min: 1, max: 32}
learning_rate = 5e-4 #@param{type:"number"}
seed = 42 #@param{type:"integer"}

seed_everything(seed)

data = DataModule(
    train_file=train_file ,
    valid_file=valid_file,
    inputs_col=inputs_col,
    outputs_col=outputs_col,
    tokenizer_name_or_path = tokenizer_name_or_path,
    input_max_length = input_max_length,
    output_max_length = output_max_length,
    batch_size = batch_size,
    num_workers = os.cpu_count(),
)

model = MT5(model_name_or_path=model_name_or_path, learning_rate=learning_rate)

Trainer(max_epochs=epochs, gpus=1, default_root_dir=output_dir).fit(model=model, datamodule=data)

# Inference

In [ ]:
from transformers import AutoTokenizer
from source.mt5 import MT5

# Do not forget to update this arg with your own model
path_to_checkpoint = 'path_to_checkpoint.ckpt'

# Loading model and tokenizer
model = MT5.load_from_checkpoint(path_to_checkpoint).eval().cuda()
model.tokenizer = AutoTokenizer.from_pretrained('tokenizer', use_fast=True)

input = 'Your input here. '

# Prediction
prediction = model.predict(inputs=[input])